In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from MetabolomicsPaper.GeneralHelperFunctions import *
# from ACSPaper.Matching import *
from ACSPaper.ACSPaperUtils import *
from sklearn.metrics import roc_curve, auc
import shap
from sklearn.model_selection import train_test_split
from statsmodels.regression.linear_model import OLS

paper_dir = '/net/mraid08/export/jafar/Microbiome/Analyses/Noamba/Cardio/ACS_study/v2/'
curr_dir = os.path.join(paper_dir, 'metacardis_SHAP')
make_dir_if_not_exists(curr_dir)
os.chdir(paper_dir)

,Blastocystis.sp..subtype.3..CAG00002.,unclassified.Oscillibacter..CAG00006_1.,unclassified.Clostridiales..CAG00006_4.,Clostridium.bolteae..CAG00008.,unclassified.Clostridiales..CAG00010_1.,Escherichia.coli..CAG00011.,Klebsiella.pneumoniae..CAG00012.,unclassified.Clostridiales..CAG00013.,Eisenbergiella.tayi..CAG00014.,Hungatella.hathewayi.2..CAG00015.,...,unclassified.Firmicutes..CAG01410.,unclassified.Oscillibacter..CAG01551.,Firmicutes.bacterium.CAG.65...Clostridium.sp..2789STDY5608883..CAG01637.,unclassified.Clostridiales..CAG01700.,Clostridium.sp..2789STDY5608793..CAG02021.,Olsenella.sp..Marseille.P2300..CAG02723.,Lactobacillus.delbrueckii..CAG03242.,unclassified.Lachnospiraceae..CAG50001.,unclassified.Clostridiales..CAG50002.,Clostridium.sp..2789STDY5834874...sp..2789STDY5608885..CAG50003.
SampleID,,,,,,,,,,,,,,,,,,,,,
x10MCx1135,0.000000,0.000000,0.0,0.292370,0.000000,10.578473,0.000000,0.000000,0.345528,0.000000,...,0.0,0.000000,33.516219,3.827387,0.983426,0.000000,0.159474,3.986862,0.000000,0.000000
x10MCx1140,0.000000,13.582625,0.0,0.805190,0.000000,1.174785,0.000000,0.000000,0.831589,0.290396,...,0.0,6.032322,6.164321,4.091947,0.000000,0.303596,0.000000,1.095586,2.204372,0.000000
x10MCx1144,0.000000,0.000000,0.0,32.570795,0.000000,84.647126,5.295063,0.541820,3.016955,10.196075,...,0.0,0.000000,0.000000,1.884057,0.000000,0.000000,0.886615,0.000000,0.000000,0.000000
x10MCx1144,0.000000,0.000000,0.0,32.570795,0.000000,84.647126,5.295063,0.541820,3.016955,10.196075,...,0.0,0.000000,0.000000,1.884057,0.000000,0.000000,0.886615,0.000000,0.000000,0.000000
x10MCx1146,0.000000,2.149893,0.0,0.131328,0.000000,0.111872,0.000000,0.087552,0.170240,0.175104,...,0.0,0.000000,10.530584,0.291841,0.000000,0.038912,0.000000,1.473795,0.384257,0.097280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
x30MCx3379,0.000000,0.081724,0.0,0.000000,1.078756,0.000000,0.000000,0.000000,0.081724,0.000000,...,0.0,0.000000,141.954507,3.432406,3.138200,0.000000,0.000000,1.977720,0.735516,2.320960
x30MCx3382,0.000000,0.000000,0.0,0.000000,2.923111,1.594424,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,20.860386,2.314130,1.350832,0.000000,0.000000,1.339759,1.306542,1.018660
x30MCx3382,0.000000,0.000000,0.0,0.000000,2.923111,1.594424,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,20.860386,2.314130,1.350832,0.000000,0.000000,1.339759,1.306542,1.018660


In [119]:
metacardis_dir = os.path.join('/net/mraid08/export/jafar/Microbiome/Analyses/Noamba/MetaCardis')
metacardis_X_path = os.path.join(metacardis_dir, 'dataframes', 'mb_diet_covariates_drugs_group.csv')
metacardis_diabetes_path = os.path.join(metacardis_dir, 'dataframes', 'MetaCardis_DiabetesRelatedVar_KC_230521.csv')
# metacardis_metabolomics_metabolon_path = os.path.join(metacardis_dir, 'metabolomics', 'metabolon', 'ShortlistedMetabolonMetabolites_rawdata_MC_forSHAP_KC_200521.csv')
# metacardis_metabolomics_nmr_path = os.path.join(metacardis_dir, 'metabolomics/robust_zs_metabolomics/proc_metabolomics.csv')
metacardis_metabolomics_path =  os.path.join(metacardis_dir, 'metabolomics', 'metabolon', 
                                             'MC_Metabolites1513datafileSHAP_forNoam_KC_030621.csv')

metacardis_X = pd.read_csv(metacardis_X_path, index_col=0).replace({'nan': np.nan})
metacardis_metadata_path = os.path.join(metacardis_dir, 'metabolomics', 'metadata', 'hub.metadata_reduced.samples.v10.r')
metacardis_X = metacardis_X.merge(pd.read_csv(metacardis_metadata_path, sep='\t', index_col=0)[['PATGROUPFINAL_C', 'SMOKE']], 
                                  on='SampleID', how='left').replace({'Yes':1, 'No':0})
metacardis_diabetes = pd.read_csv(metacardis_diabetes_path, index_col=0).replace({'Yes':1, 'No':0})
metacardis_diabetes.index.names = ['SampleID']
metacardis_X_age_bmi = metacardis_X[['AGE', 'BMI_C', 'Group', 'GENDER', 'PATGROUPFINAL_C', 'SMOKE']].join(metacardis_diabetes).copy()
# metacardis_metabolon_metabolomics = pd.read_csv(metacardis_metabolomics_metabolon_path, index_col=0)
# metacardis_metabolon_metabolomics = metacardis_metabolon_metabolomics.rename(columns={k:k.replace('9999', 'X - ') for k in metacardis_metabolon_metabolomics.columns})
# metacardis_metabolon_metabolomics = metacardis_metabolon_metabolomics.drop(['Status', 'MC_ID'], axis=1).groupby('ID').first()
# metacardis_metabolon_metabolomics = metacardis_metabolon_metabolomics.apply(lambda x: (x-x.median())/x.std())
# metacardis_nmr_metabolomics = pd.read_csv(metacardis_metabolomics_nmr_path, index_col=0)
# metacardis_nmr_metabolomics.index.names = ['ID']
# metacardis_nmr_metabolomics = metacardis_nmr_metabolomics.loc[:, ['urine' not in s.split('___')[0] for s in metacardis_nmr_metabolomics.columns]]
# metacardis_serum_metabolomics = pd.concat((metacardis_metabolon_metabolomics, metacardis_nmr_metabolomics), axis=1)

metacardis_serum_metabolomics = pd.read_csv(metacardis_metabolomics_path, index_col=0).dropna(how='all', axis=0).dropna(how='all', axis=1).drop(['Status'], axis=1)
metacardis_serum_metabolomics.index.names = ['SampleID']
metacardis_serum_metabolomics = metacardis_serum_metabolomics.groupby('SampleID').first()
metacardis_serum_metabolomics = metacardis_serum_metabolomics.apply(lambda x: (x-x.median())/x.std())


diet = metacardis_X.filter(regex='food|nutr')
diet.to_csv(os.path.join(curr_dir, 'diet.csv'))

drugs = pd.read_csv(os.path.join(metacardis_dir, 'dataframes', 'drugs_data.csv'), index_col=0).groupby('SampleID').first()
drugs.to_csv(os.path.join(curr_dir, 'drugs.csv'))

mb = pd.read_csv(os.path.join(metacardis_dir, 'metagenomics', 'MC_MGSdatafileforSHAP_forNoam_KC_030621.csv'), index_col=0)
mb = mb.drop(['Status'], axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
mb.index.names = ['SampleID']
mb = mb.groupby('SampleID').first()
mb.to_csv(os.path.join(curr_dir, 'mb.csv'))

full_index = list(set(metacardis_serum_metabolomics.index).intersection(set(metacardis_X_age_bmi.index)))
metacardis_serum_metabolomics, metacardis_X_age_bmi = metacardis_serum_metabolomics.loc[full_index].astype(float), metacardis_X_age_bmi.loc[full_index]

age_sex_group_smoke_diabetes = metacardis_X_age_bmi.drop(['Status', 'PATGROUPFINAL_C'], axis=1)
age_sex_group_smoke_diabetes = age_sex_group_smoke_diabetes.join(pd.get_dummies(age_sex_group_smoke_diabetes['Group'])).drop(['Group'], axis=1)
age_sex_group_smoke_diabetes.to_csv(os.path.join(curr_dir, 'age_sex_group_smoke_diabetes.csv'))

metacardis_serum_metabolomics.to_csv(os.path.join(curr_dir, 'metacardis_serum_metabolomics.csv'))

mb.join(diet).join(drugs).join(age_sex_group_smoke_diabetes).to_csv(os.path.join(curr_dir, 'X.csv'))

metacardis_X_age_bmi.Group.value_counts()

IHD372     319
HC275      275
MMC269     218
UMCC222    211
Name: Group, dtype: int64

command

python ~/workspace/metabolon/Analyses/RSCV.py /net/mraid08/export/jafar/Microbiome/Analyses/Noamba/Cardio/ACS_study/v2/metacardis_SHAP/metabolomics1513_fullX/ -path_to_X /net/mraid08/export/jafar/Microbiome/Analyses/Noamba/Cardio/ACS_study/v2/metacardis_SHAP/X.csv -path_to_Y /net/mraid08/export/jafar/Microbiome/Analyses/Noamba/Cardio/ACS_study/v2/metacardis_SHAP/metacardis_serum_metabolomics.csv -n_random 5 -k_folds 5 -job_name MC-SHAP &

# results

In [123]:
results = pd.read_csv(os.path.join(curr_dir, 'metabolomics1513_fullX', 'results.csv'), index_col=0)

In [159]:
results.sort_values('Coefficient_of_determination').tail(80)

,Size,Coefficient_of_determination,explained_variance_score,pearson_r,pearson_p,spearman_r,spearman_p
sphingomyelin (d18:2/24:2)*,995.0,0.315398,0.315399,0.562343,4.846699e-84,0.591764,4.793960e-95
X-12216,995.0,0.315429,0.315609,0.561833,7.354601e-84,0.519699,6.317532e-70
1-(1-enyl-palmitoyl)-2-linoleoyl-GPC (P-16:0/18:2)*,995.0,0.315707,0.315715,0.562509,4.228909e-84,0.575286,9.579265e-89
docosahexaenoylcholine,995.0,0.316124,0.316184,0.562371,4.735749e-84,0.569063,1.858894e-86
N-acetylcarnosine,995.0,0.316608,0.316611,0.564005,1.238477e-84,0.524761,1.693222e-71
...,...,...,...,...,...,...,...
alpha-hydroxymetoprolol,995.0,0.565259,0.565298,0.753989,1.966692e-183,0.543181,1.902349e-77
hydroxycotinine,995.0,0.604708,0.604892,0.778285,6.675398e-203,0.485606,5.299979e-60
salicyluric glucuronide*,995.0,0.621672,0.621683,0.789384,1.221064e-212,0.692418,6.153992e-143
metformin,995.0,0.639302,0.639331,0.801272,9.547890e-224,0.461144,1.527990e-53


In [128]:
abs_signed_shap = pd.read_csv(os.path.join(curr_dir, 'metabolomics1513_fullX', 'abs_signed_shap.csv'), index_col=0)